# Spacy/Medspacy IAA

## Resources

Prodigy forum answer about IAA for spans https://support.prodi.gy/t/proper-way-to-calculate-inter-annotator-agreement-for-spans-ner/5760

Spacy scorer object https://spacy.io/api/scorer

## End Goal

### Functionality

Provide a collection of methods to evaluate IAA between _n_ arbitrary spacy `doc` objects. Provide methods that aid in error analysis such as providing lists of differences.

Priorities:
* Pairwise F1
    * configurable strict/loose matching
    * configurable inclusion of labels/attributes (calculate just span vs span+class agreement)

* Imported python files
    * reasonable docstrings on methods/classes
    
* Unit tests
    * add CI to repo for automated testing later

Extra features:
* List of differences between docs
* 

Expected challenges
* Spacy scorer functions are useful, but _only_ do strict span matching
* Fewer resources (obviously?) available for comparisons between 3+ docs


In [2]:
import spacy

In [3]:
nlp = spacy.load("en_core_web_sm")

In [7]:
doc = nlp("this is a test document made in utah.")

In [8]:
doc.ents

(utah,)

In [9]:
doc2 = nlp("this is a test document made in utah.")

In [11]:
ents = doc.ents

In [ ]:
def overlaps(ent1, ent2):
    '''calculate whether two ents overlap
    returns bool
    '''

In [ ]:
def exact_match(ent1, ent2):
    '''calculate whether two ents have exact overlap
    returns bool
    '''

In [12]:
def agreement(doc1, doc2, fuzzy):
    '''calculate the agreement betweent two docs
       returns confusion matrix
    '''
    tp = 0
    fp = 0
    fn = 0
    tn = 0
    for ent1 in doc1:
        for ent2 in doc2
            if loose:
                if overlaps(ent1, ent2)
                    tp++
                else:
                    fn++
            else
                exact_match
    return (tp, fp, fn, tn)

In [13]:
def pairwise_f1(confusion matrix):
    '''calculate f1 with given true positive, false positive, and false negative values'''
    
    return f1

In [14]:
def corpus_agreement(docs1, docs2, fuzzy):
    '''calculate f1 over an entire corpus of documents'''
    
    corpus_confusion_matrix = ()
    
    for i, doc1 in enumerate(docs1):
        agreement(doc1, docs2[i])
    
    return corpus_confusion_matrix

SyntaxError: invalid syntax (654967868.py, line 6)